In [ ]:
!pip install -q autogluon.tabular
!pip install -q ray==2.10.0

In [ ]:
from autogluon.tabular import TabularPredictor
import warnings
import shutil

warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
TARGET = 'class'
TIME_LIMIT = 3600 * 10

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')

In [ ]:
add_data1 = pd.read_csv('/kaggle/input/previous-dataset/1987_data_no_miss_modify.csv')

In [ ]:
train = pd.concat([train, add_data1])

In [ ]:
train = train.drop(columns=['id'], axis=1)
test = test.drop(columns=['id'], axis=1)

In [ ]:
def remove_duplicates(df):
    duplicates = df[df.duplicated()]
    df = df.drop_duplicates()
    return df, duplicates

In [ ]:
train, duplicates = remove_duplicates(train)
print(f"Removed {len(duplicates)} duplicate rows.")

In [ ]:
train.reset_index(drop=True, inplace=True)

## Remove Wrong Data

In [ ]:
object_cols = test.select_dtypes(include='object').columns
object_cols

In [ ]:
import re

def extract_single_alpha(input_string):
    # check is string or nan
    if not isinstance(input_string, str) or input_string == np.nan:
        return np.nan

    # Step 1: Remove all non-alphabet characters (numbers, spaces, special characters)
    cleaned_string = re.sub(r'[^a-zA-Z\s]', '', input_string) # remain ' '

    # Step 2: Split the cleaned string by spaces to separate words
    parts = cleaned_string.split()

    # Step 3: Filter out only those parts that are single alphabet characters
    single_char_alphabets = [part for part in parts if len(part) == 1 and part.isalpha()]

    # Step 4: Return the first single-character alphabet if it exists
    if single_char_alphabets:
        return single_char_alphabets[0]
    else:
        return np.nan
    
for col in object_cols:
    train[col] = train[col].apply(extract_single_alpha)
    test[col] = test[col].apply(extract_single_alpha)

---

In [ ]:
predictor = TabularPredictor(
    label=TARGET,
    eval_metric='mcc',
    problem_type='binary',
    verbosity=2
)

In [ ]:
predictor.fit(
    train_data=train,
    time_limit=TIME_LIMIT,
    presets='best_quality',
    excluded_model_types=['KNN'],
    ag_args_fit={
        'num_gpus': 1, 
        'stopping_metric': 'log_loss'
    }
)

In [ ]:
predictor.leaderboard()

In [ ]:
test_preds = predictor.predict(test)

In [ ]:
sub = pd.read_csv('/kaggle/input/playground-series-s4e8/sample_submission.csv')
sub[TARGET] = test_preds.values
sub.to_csv('submission.csv', index=False)
sub.head()

In [ ]:
shutil.rmtree("AutogluonModels")